## code to convert precipitation into normal hourly values (NB units!)

In [1]:
# packages
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

import glob


In [2]:
# read the data 

path = '/Volumes/Data/Share/plaza/4varya/precip/'
file = xr.open_dataset(path + 'total_precip_2020-22.nc')

FileNotFoundError: [Errno 2] No such file or directory: b'/Volumes/Data/Share/plaza/4varya/precip/total_precip_2020-22.nc'

In [21]:
''' these are not nesesary, i did that to speed up the calcultions'''
# select location
gridcell = file.sel(latitude=55, longitude=55, method='nearest')

# gridcell = file 

# select time
start_date = '2020-06-01T00:00:00.000000000'
end_date = '2020-06-06T00:00:00.000000000'
cut = gridcell.sel(time=slice(start_date, end_date))


NameError: name 'file' is not defined

In [ ]:
# look at the data:
# convert to dataframe for convenience 
cutdf = cut.to_dataframe()

# look at the dataframe
cutdf.head()

In [ ]:
# # look at the data: plot
# # plt.figure(figsize=(12, 6))
# cut['tp'].plot(x = 'time')#.line(x='time', marker='o')
# plt.grid(True)


In [ ]:
''' you see that the precip accumulates and then resets in the end of each day, but what time does that happen - the max is in midnight'''

print('max precip time:', cutdf.tp.idxmax())


In [ ]:
''' we have to shift the precipitation one "step" back, so that max value occurs at 23:00, so that it is the same day '''

shifted = cut.shift(time= -1, fill_value=0)

In [ ]:
''' calculate the differences per hour within each day using shifted data ''' 

# Create a sample xarray dataset for a year of data
# data = shifted

# Function to calculate hourly differences with reset at midnight
def calculate_hourly_diff(arr):
    diff = np.diff(arr, prepend=0)
    return xr.DataArray(diff, coords=arr.coords)

# Calculate hourly differences with reset at midnight for the entire time series
hourly_values = shifted.tp.groupby('time.dayofyear').apply(calculate_hourly_diff)

# Create a new dataset with the hourly differences
hourly_data = xr.Dataset({'hourly_values': hourly_values})


In [ ]:
# a = hourly_data.isel(time = 1).hourly_values.plot()


In [ ]:
''' plot to see if its correct '''
plt.figure(figsize=(20, 6))
shifted['tp'].plot.line(x='time', marker='o', color ='lightblue', label ='original')
hourly_data['hourly_values'].plot.line(x='time', marker='o', color ='orange', label = 'hourly')

plt.grid()
plt.legend()

In [ ]:
''' calculate the sum within day in a shifted data, compare with the max within the same day in the original data '''

# slice data - take a day  
start_date = '2020-01-01T00:00:00.000000000'
end_date = '2020-01-01T23:00:00.000000000'

# Slice the dataset for a day 
cut_sh = shifted.sel(time=slice(start_date, end_date))
cut_hourl = hourly_data.sel(time=slice(start_date, end_date))

print('new hourly data - daily sum:', round(cut_hourl.hourly_values.values.sum(), 6))
print('original data - daily max (cumulative sum):', round(cut_sh.tp.values.max(), 6))

''' correct! '''

In [ ]:
''' since we shifted data, the last data point doesnt make sense anymore - cut it out''' 
hourly_data = hourly_data.isel(time=slice(None, -1))


# hourly_diff_data - is the final dataset!! 

## same but for csv


In [ ]:
# read csv 




# new stuff! 


In [14]:
# read all data

directory = '/Users/varyabazilova/Desktop/uu/0_teaching/Msc_students/Leon/testera5land/ncfiles/'
files = glob.glob(directory + '*.nc')

# Use xarray.open_mfdataset to open and concatenate the .nc files
data = xr.open_mfdataset(files)
# data['tp'] = data.tp * 1000
data = data.sortby('time')

data = data.shift(time= -1, fill_value=0).sortby('time')


In [15]:
# Calculate the hourly differences within each day
def calculate_hourly_diff(arr):
    diff = arr - arr.shift(time=1, fill_value=0)
    return xr.DataArray(diff, coords=arr.coords)

In [16]:
hourly_values = data.tp.groupby('time.dayofyear').apply(calculate_hourly_diff)
hourly_data = xr.Dataset({'hourly_values': hourly_values})


## check 

In [17]:
''' calculate the sum within day in a shifted data, compare with the max within the same day in the original data '''
# slice coordinates
data = data.sel(latitude=29.0, longitude=84.0, method='nearest')
hourly_data = hourly_data.sel(latitude=29, longitude=84, method='nearest')


In [18]:
# slice data - take a day  
start_date = '2020-01-01T00:00:00.000000000'
end_date = '2020-01-01T23:00:00.000000000'

# Slice the dataset for a day 
cut_sh = data.sel(time=slice(start_date, end_date))
cut_hourl = hourly_data.sel(time=slice(start_date, end_date))

print('new hourly data - daily sum:', round(cut_hourl.hourly_values.values.sum(), 6))
print('original data - daily max (cumulative sum):', round(cut_sh.tp.values.max(), 6))

# ''' correct! '''

new hourly data - daily sum: 0.000613
original data - daily max (cumulative sum): 0.000613
